# Global Average Precision

### Kaggle Google Landmark Recognition Challenge
https://www.kaggle.com/c/landmark-recognition-challenge#evaluation

Submissions are evaluated using Global Average Precision (GAP) at k, where k=1. This metric is also known as micro Average Precision (microAP), as per [1]. It works as follows:

For each query image, you will predict one landmark label and a corresponding confidence score. The evaluation treats each prediction as an individual data point in a long list of predictions (sorted in descending order by confidence scores), and computes the Average Precision based on this list.

If a submission has N predictions (label/confidence pairs) sorted in descending order by their confidence scores, then the Global Average Precision is computed as:

- $GAP=\frac{1}{M}\sum_{i=1}^{N}P(i)rel(i)$
- where:
- $N$ is the total number of predictions returned by the system, across all queries
- $M$ is the total number of queries with at least one landmark from the training set visible in it (note that some queries may not depict landmarks)
- $P(i)$ is the precision at rank $i$
- $rel(i)$ denotes the relevance of prediciton $i$: it’s 1 if the $i$-th prediction is correct, and 0 otherwise

[1] F. Perronnin, Y. Liu, and J.-M. Renders, "A Family of Contextual Measures of Similarity between Distributions with Application to Image Retrieval," Proc. CVPR'09

- ref : https://www.kaggle.com/davidthaler/gap-metric

In [63]:
import numpy as np
import pandas as pd

def GAP_vector(pred, conf, true, return_x=False):
    '''
    Compute Global Average Precision (aka micro AP), the metric for the
    Google Landmark Recognition competition. 
    This function takes predictions, labels and confidence scores as vectors.
    In both predictions and ground-truth, use None/np.nan for "no label".

    Args:
        pred: vector of integer-coded predictions
        conf: vector of probability or confidence scores for pred
        true: vector of integer-coded labels for ground truth
        return_x: also return the data frame used in the calculation

    Returns:
        GAP score
    '''
    x = pd.DataFrame({'pred': pred, 'conf': conf, 'true': true})
    x.sort_values('conf', ascending=False, inplace=True, na_position='last')
    x['correct'] = (x.true == x.pred).astype(int)
    x['prec_k'] = x.correct.cumsum() / (np.arange(len(x)) + 1)
    x['term'] = x.prec_k * x.correct
    gap = x.term.sum() / x.true.count()
    if return_x:
        return gap, x
    else:
        return gap

1. confidence 를 기준으로 정렬
2. correct 여부 표시
3. x['prec_k'] = x.correct.cumsum() 계산
4. x['term'] = x.prec_k * x.correct
5. gap = x.term.sum() / 데이터 수

In [73]:
image_id = np.random.randint(0, 100, size=10)
ypred = np.random.choice([1,2,3], 10)
ytrue = np.random.choice([1,2,3], 10)
conf = np.random.random(10)

In [74]:
image_id

array([48, 60, 21, 35, 42, 48, 16, 15, 88, 65])

In [75]:
ypred

array([3, 2, 3, 2, 3, 1, 1, 2, 2, 2])

In [76]:
ytrue

array([3, 3, 1, 3, 1, 2, 1, 2, 1, 1])

In [77]:
df = pd.DataFrame({'image_id': image_id, 'ytrue': ytrue, 'ypred': ypred, 'conf': conf})

In [78]:
df['correct'] = (df.ypred == df.ytrue).astype(int)
df.sort_values('conf', ascending=False, inplace=True, na_position='last')
df['pred_k'] = df.correct.cumsum()
df['term'] = df.pred_k * df.correct
gap = df.correct.sum() / len(df)

In [81]:
df.sort_values('image_id')

,conf,image_id,ypred,ytrue,correct,pred_k,term
7,0.403362,15,2,2,1,2,2
6,0.506572,16,1,1,1,1,1
2,0.089852,21,3,1,0,3,0
3,0.304743,35,2,3,0,2,0
4,0.501004,42,3,1,0,1,0
5,0.251091,48,1,2,0,2,0
0,0.159241,48,3,3,1,3,3
1,0.639684,60,2,3,0,0,0
9,0.862079,65,2,1,0,0,0
8,0.359474,88,2,1,0,2,0


In [82]:
gap

0.29999999999999999

제출을 위한 파일은 만드는건 ypred와 confidence만 연결하면 된다.